# side model feature selection

In [1]:
import pandas as pd

df_features = pd.read_parquet("data/features.parquet")
print(df_features.shape)
df_features.isna().sum(axis=0).sort_values(ascending=False)

(13219, 3631)


adx_7                     0
mod_rsi_ddt               0
ma_difference_dt_lag15    0
ma_difference_dt_lag16    0
ma_difference_dt_lag17    0
                         ..
conv_8                    0
conv_9                    0
conv_10                   0
conv_11                   0
williams_r_ddt_lag19      0
Length: 3631, dtype: int64

In [2]:
import numpy as np

label = np.load("data/side_label.npy")

print(label.shape)

len_gap = len(label) - len(df_features)

label = label[len_gap:]

print(label.shape)

(15059,)
(13219,)


In [3]:
from custom_indicators.toolbox.feature_selection.rfcq_selector import RFCQSelector

selector = RFCQSelector()
selector.fit(df_features, label)

➤ 识别数值型变量...
➤ 计算特征与目标变量的相关性(使用随机森林)...
✓ 选择第1个特征: voss (最大重要性: 0.0421)
➤ 计算特征冗余度...
➤ 总计选择726个特征 (已选择1个，还需选择725个)...
➤ 开始MRMR迭代选择过程...


选择特征:   0%|                                                           | 0/725 [00:00<?, ?特征/s]


✅ 特征选择完成：从3631个特征中选择了726个，舍弃了2905个
✅ 选择的特征: ['voss', 'corwin_schultz_estimator_ddt', 'acc_swing_index_lag9', 'stc_ddt', 'reactivity', 'evenbetter_sinewave_short_dt', 'adx_7_dt', 'natr', 'vwap_dt', 'stc_dt', 'ehlers_early_onset_trend_ddt_lag1', 'ehlers_early_onset_trend_ddt', 'fisher_dt', 'vmd_win256_0', 'acc_swing_index_dt_lag1', 'natr_dt', 'cwt_win32_6', 'voss_lag1', 'vwap_dt_lag1', 'williams_r', 'vmd_win64_0', 'vwap_dt_lag2', 'cmma', 'fisher_dt_lag2', 'stc_dt_lag1', 'bekker_parkinson_vol', 'bandpass', 'forecast_oscillator_lag2', 'roofing_filter_ddt_lag1', 'aroon_diff', 'natr_ddt', 'trendflex_dt', 'cmma_lag1', 'amihud_lambda_lag2', 'bandpass_dt', 'acc_swing_index_dt', 'trendflex_dt_lag1', 'evenbetter_sinewave_short', 'fisher_ddt_lag1', 'ehlers_early_onset_trend_dt', 'aroon_diff_dt', 'cwt_win256_6', 'adaptive_cci_dt', 'swamicharts_stochastic_2', 'acc_swing_index_lag2', 'evenbetter_sinewave_long', 'voss_filt_dt', 'fisher_dt_lag1', 'cwt_win1024_5', 'roofing_filter_ddt_lag2', 'cwt_win128

In [4]:
side_res_long = pd.Series(selector.relevance_, index=selector.variables_).sort_values(
    ascending=False
)
print((side_res_long > 0).sum())
side_res_long

161


voss                            0.042143
stc_dt                          0.033517
vwap_dt                         0.031316
fisher_dt                       0.030990
evenbetter_sinewave_short_dt    0.029825
                                  ...   
conv_25                         0.000000
conv_26                         0.000000
conv_27                         0.000000
conv_28                         0.000000
williams_r_ddt_lag19            0.000000
Length: 3631, dtype: float64

In [5]:
import json

feature_info = {
    "side": {
        "side": [],
    },
    "meta": {
        "meta": [],
        "model": [],
    },
}

feature_long = side_res_long[side_res_long > 0].index.tolist()


feature_info["side"]["side"] = feature_long


with open("strategies/BinanceBtcEntropyBarV1/feature_info.json", "w") as f:
    json.dump(feature_info, f, indent=4)

# meta model feature selection

In [1]:
import numpy as np
import pandas as pd

df_features = pd.read_parquet("data/features.parquet")
meta_label = np.load("data/label_meta.npy")
print(df_features.shape)
print(meta_label.shape)

df_features.isna().sum(axis=0).sort_values(ascending=False)

(13219, 3631)
(13219,)


adx_7                     0
mod_rsi_ddt               0
ma_difference_dt_lag15    0
ma_difference_dt_lag16    0
ma_difference_dt_lag17    0
                         ..
conv_8                    0
conv_9                    0
conv_10                   0
conv_11                   0
williams_r_ddt_lag19      0
Length: 3631, dtype: int64

In [2]:
from strategies.BinanceBtcEntropyBarV1.config import (
    SIDE,
    get_side_model,
)

side_model = get_side_model(False)

side_res = side_model.predict(df_features[SIDE])

assert df_features.shape[0] == len(side_res)

df_features["model"] = side_res

/opt/homebrew/Caskroom/miniconda/base/envs/jesse/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [3]:
# meta feature selection
from custom_indicators.toolbox.feature_selection.rfcq_selector import RFCQSelector

selector = RFCQSelector()
selector.fit(df_features, meta_label)
meta_res = pd.Series(selector.relevance_, index=selector.variables_).sort_values(
    ascending=False
)

➤ 识别数值型变量...
➤ 计算特征与目标变量的相关性(使用随机森林)...
✓ 选择第1个特征: model (最大重要性: 0.0361)
➤ 计算特征冗余度...
➤ 总计选择726个特征 (已选择1个，还需选择725个)...
➤ 开始MRMR迭代选择过程...


选择特征:   0%|                                                           | 0/725 [00:00<?, ?特征/s]


✅ 特征选择完成：从3632个特征中选择了726个，舍弃了2906个
✅ 选择的特征: ['model', 'fti_lag17', 'sample_entropy_win512_array', 'iqr_ratio_dt', 'cmma_lag15', 'vwap_lag4', 'dft_spectrum_9', 'hurst_coef_slow_lag11', 'phase_accumulation_dt', 'amihud_lambda_lag16', 'evenbetter_sinewave_long_ddt', 'conv_26', 'sample_entropy_win256_spot', 'fti_lag13', 'amihud_lambda_lag2', 'swamicharts_rsi_27', 'approximate_entropy_win256_spot', 'dft_dom_cycle_lag16', 'fti_ddt_lag16', 'adx_7_dt', 'vwap_lag17', 'price_variance_ratio_lag13', 'ac_15', 'kyle_lambda_lag11', 'hurst_coef_slow_lag12', 'cmma_lag9', 'fti_lag5', 'cwt_win256_20', 'phase_accumulation_dt_lag10', 'dft_spectrum_3', 'amihud_lambda_lag11', 'kyle_lambda_lag9', 'hurst_coef_fast_dt_lag1', 'hurst_coef_slow_lag9', 'adx_7_lag10', 'acc_swing_index_lag8', 'swamicharts_stochastic_25', 'phase_accumulation_lag11', 'adaptive_cci_lag12', 'amihud_lambda_lag1', 'sample_entropy_win64_spot', 'cwt_win32_19', 'fti_lag15', 'dual_diff_lag1', 'corwin_schultz_estimator', 'amihud_lambda_lag15',

In [4]:
(meta_res > 0).sum()

204

In [5]:
import json
from pathlib import Path

feature_info_path = Path("strategies/BinanceBtcEntropyBarV1/feature_info.json")
if feature_info_path.exists():
    with open(feature_info_path, "r") as f:
        feature_info = json.load(f)

feature_info["meta"]["meta"] = []
feature_info["meta"]["model"] = []

meta_features = meta_res[meta_res > 0].index.tolist()
for k in meta_features:
    if k == "model":
        feature_info["meta"]["model"].append(k)
    else:
        feature_info["meta"]["meta"].append(k)


with open("strategies/BinanceBtcEntropyBarV1/feature_info.json", "w") as f:
    json.dump(feature_info, f, indent=4)